In [1]:
%load_ext autoreload
%autoreload 2
import os
if not os.path.exists('./modules') and not os.path.exists('modules.zip'):
    from google.colab import files
    uploaded = files.upload()
if not os.path.exists('./modules') and os.path.exists('modules.zip'):
    os.system('unzip modules.zip -d .')

if not os.path.exists('./Models') and not os.path.exists('Models.zip'):
    from google.colab import files
    uploaded = files.upload()
if not os.path.exists('./Models') and os.path.exists('Models.zip'):
    os.system('unzip Models.zip -d .')

!pip3 install optuna
!pip3 install mne
import torch
import torch.nn as nn
import optuna
from modules import Trainer
from modules.competition_dataset_one_per_trial import EEGDataset
from modules.utils import evaluate_model
import random
import numpy as np
from torch.utils.data import DataLoader
from Models.lfann_dann_gpt_generated import LFANN_DANN

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip
c:\Users\ahmad\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\ahmad\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


device(type='cuda')

In [2]:
# from google.colab import drive

# drive.mount("/content/drive/")
# try:
#     print()
#     os.symlink("/content/drive/MyDrive/ai_data/eeg_detection/data", "./data")
#     os.symlink("/content/drive/MyDrive/ai_data/eeg_detection/checkpoints", "./checkpoints")
# except:
#     pass

data_path = "./data/mtcaic3"
model_path = "./checkpoints/mi/models/the_honored_one.pth"
optuna_db_path = "./checkpoints/mi/optuna/the_honored_one.db"

In [3]:
# Add this at the beginning of your notebook, after imports
def set_random_seeds(seed=42):
    """Set random seeds for reproducibility"""

    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

# Call this function before creating datasets and models
set_random_seeds(42)

In [4]:
batch_size = 32
eeg_channels = [
    "C3", # 2296.15
    "PZ", # 1744.43
    "C4", # 1556.46
    # "OZ" # 444.98
    # "PO7" # 381.63
    # "PO8" # 275.78
    # "CZ" # 200.43
    # "FZ" # 111.51
]

dataset_train = EEGDataset(
    data_path=data_path,
    task="MI",
    split="train",
    tmin=0.5,  # skip first 0.5 s (125 samples)
    win_len=int(4.5 * 250),  # 4.5 s window → 1125 samples
    read_labels=True,
    eeg_channels=eeg_channels,
)

dataset_val = EEGDataset(
    data_path=data_path,
    task="MI",
    split="validation",
    tmin=0.5,
    win_len=int(4.5 * 250),
    read_labels=True,
    eeg_channels=eeg_channels,
)

train_loader = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
val_loader   = DataLoader(dataset_val,   batch_size=batch_size)

np shape:  (2400, 3, 40, 1125)
data shape: torch.Size([2400, 3, 40, 1125]), classes shape: torch.Size([2400, 2])
np shape:  (50, 3, 40, 1125)
data shape: torch.Size([50, 3, 40, 1125]), classes shape: torch.Size([50, 2])
np shape:  (50, 3, 40, 1125)
data shape: torch.Size([50, 3, 40, 1125]), classes shape: torch.Size([50, 2])


In [5]:
dataset_train[0][0].shape

[autoreload of modules.competition_dataset_one_per_trial failed: Traceback (most recent call last):
  File "C:\Users\ahmad\AppData\Roaming\Python\Python312\site-packages\IPython\extensions\autoreload.py", line 280, in check
    elif self.deduper_reloader.maybe_reload_module(m):
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\ahmad\AppData\Roaming\Python\Python312\site-packages\IPython\extensions\deduperreload\deduperreload.py", line 533, in maybe_reload_module
    new_source_code = f.read()
                      ^^^^^^^^
  File "c:\Users\ahmad\AppData\Local\Programs\Python\Python312\Lib\encodings\cp1252.py", line 23, in decode
    return codecs.charmap_decode(input,self.errors,decoding_table)[0]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
UnicodeDecodeError: 'charmap' codec can't decode byte 0x90 in position 6483: character maps to <undefined>
]


torch.Size([3, 40, 1125])

In [6]:
C, F, T = dataset_train[0][0].shape
print(C, F, T)
model = LFANN_DANN(C=C, F=F, T=T).to(device)
inp = torch.randn(3, C, F, T).to(device)
out = model(inp)
out

3 40 1125


(tensor([[ 0.0040, -0.0810],
         [ 0.0041, -0.0838],
         [ 0.0008, -0.0759]], device='cuda:0', grad_fn=<AddmmBackward0>),
 tensor([[ 2.1628e-01, -9.6212e-02,  1.7485e-01, -1.3373e-01,  1.8526e-01,
           2.1715e-01,  1.8716e-01,  2.0344e-01,  2.4808e-01, -1.5374e-01,
          -2.1584e-01, -2.5827e-01, -3.6120e-01, -1.2083e-01, -4.1747e-01,
          -1.2597e-01, -3.7014e-01,  4.1168e-01,  8.7177e-02,  4.7319e-01,
           2.5332e-01,  2.6611e-01, -1.4398e-01, -1.6633e-01, -2.9214e-01,
          -3.3820e-01, -8.4273e-02,  1.9428e-01, -2.0986e-03, -2.0291e-02],
         [ 2.1674e-01, -9.6597e-02,  1.7417e-01, -1.3452e-01,  1.8650e-01,
           2.1595e-01,  1.8649e-01,  2.0159e-01,  2.4985e-01, -1.5413e-01,
          -2.1684e-01, -2.5881e-01, -3.6280e-01, -1.2203e-01, -4.1908e-01,
          -1.2603e-01, -3.7116e-01,  4.1253e-01,  8.7238e-02,  4.7475e-01,
           2.5421e-01,  2.6541e-01, -1.4459e-01, -1.6665e-01, -2.9350e-01,
          -3.3918e-01, -8.3659e-02,  1.945

In [7]:
avg_losses_label = []
avg_losses_domain = []
val_label_accuracies = []
val_domain_accuracies = []
train_label_accuracies = []
train_domain_accuracies = []

In [8]:
# try:
#     model.load_state_dict(torch.load(model_path, weights_only=True))
# except Exception:
#     print("skipping model loading...")


opt = torch.optim.Adam(model.parameters(), lr=3e-5, betas=(0.9, 0.999))
criterion = nn.CrossEntropyLoss()
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    opt,
    mode="min",  # “min” if you want to reduce lr when the quantity monitored has stopped decreasing
    factor=0.5,  # new_lr = lr * factor
    patience=20,  # number of epochs with no improvement after which lr will be reduced
    threshold=1e-4,  # threshold for measuring the new optimum, to only focus on significant changes
    threshold_mode="rel",  # `'rel'` means compare change relative to best value. Could use `'abs'`.
    cooldown=0,  # epochs to wait before resuming normal operation after lr has been reduced
    min_lr=1e-6,  # lower bound on the lr
)

epochs = 4000

for epoch in range(epochs):
    avg_loss_label = 0
    avg_loss_domain = 0
    correct_label = 0
    correct_domain = 0
    total = 0
    model.train()

    for x, y in train_loader:
        x = x.to(device)
        y = y.to(device).to(torch.int64) # shape: [Bx2], 0: label, 1: domain
        y_labels = y[:, 0]
        y_subj = y[:, 1]

        y_pred_labels, y_pred_domain = model(x)

        loss_label = criterion(y_pred_labels, y_labels)
        loss_domain = criterion(y_pred_domain, y_subj)
        loss = loss_label + loss_domain
        
        opt.zero_grad()
        loss.backward()
        opt.step()

        avg_loss_label += loss_label.item()
        avg_loss_domain += loss_domain.item()

        # Accuracy calculation
        _, pred_labels = torch.max(y_pred_labels, 1)
        _, pred_domains = torch.max(y_pred_domain, 1)
        correct_label += (pred_labels == y_labels).sum().item()
        correct_domain += (pred_domains == y_subj).sum().item()
        total += y_labels.size(0)

    avg_loss_label /= len(train_loader)
    avg_loss_domain /= len(train_loader)
    avg_losses_label.append(avg_loss_label)
    avg_losses_domain.append(avg_loss_domain)
    train_label_acc = 100.0 * correct_label / total
    train_domain_acc = 100.0 * correct_domain / total
    train_label_accuracies.append(train_label_acc)
    train_domain_accuracies.append(train_domain_acc)
    # scheduler.step(avg_loss_label)

    if epoch % 5 == 0:
        label_evaluation, domain_evaluation = evaluate_model(model, val_loader, device)
        val_label_accuracies.append(label_evaluation)
        val_domain_accuracies.append(domain_evaluation)
        model.cpu()
        torch.save(model.state_dict(), model_path)
        model.to(device)
        print(f"{epoch}: avg_loss_label: {avg_loss_label:.3F}, avg_loss_domain: {avg_loss_domain:.3F}, train_label_acc: {train_label_acc:.2f}%, train_domain_acc: {train_domain_acc:.2f}%, label_evaluation: {(label_evaluation*100):.2f}, domain_evaluation: {(domain_evaluation*100):.2f}, lr: {opt.param_groups[0]['lr']}")

0: avg_loss_label: 0.695, avg_loss_domain: 3.432, train_label_acc: 49.25%, train_domain_acc: 3.33%, label_evaluation: 56.00, domain_evaluation: 0.00, lr: 3e-05
5: avg_loss_label: 0.694, avg_loss_domain: 3.453, train_label_acc: 49.46%, train_domain_acc: 3.33%, label_evaluation: 60.00, domain_evaluation: 0.00, lr: 3e-05
10: avg_loss_label: 0.695, avg_loss_domain: 3.489, train_label_acc: 49.67%, train_domain_acc: 6.54%, label_evaluation: 60.00, domain_evaluation: 0.00, lr: 3e-05
15: avg_loss_label: 0.696, avg_loss_domain: 4.375, train_label_acc: 51.17%, train_domain_acc: 4.29%, label_evaluation: 60.00, domain_evaluation: 0.00, lr: 3e-05
20: avg_loss_label: 0.704, avg_loss_domain: 24.994, train_label_acc: 50.12%, train_domain_acc: 4.62%, label_evaluation: 48.00, domain_evaluation: 0.00, lr: 3e-05
25: avg_loss_label: 2.158, avg_loss_domain: 73.371, train_label_acc: 50.21%, train_domain_acc: 1.50%, label_evaluation: 48.00, domain_evaluation: 0.00, lr: 3e-05
30: avg_loss_label: 1.734, avg_los

KeyboardInterrupt: 

In [ ]:
class CustomTrainer(Trainer):
    # This method is called by _objective during an Optuna trial
    def prepare_trial_run(self):
        assert isinstance(self.trial, optuna.Trial), "Trial not set!"

        # 1. Define Hyperparameters for this trial
        #    a. Data/Loader parameters
        window_length = self.trial.suggest_categorical("window_length", [128, 256, 640]) # e.g. 64*2, 64*4, 64*10
        batch_size = self.trial.suggest_categorical("batch_size", [32, 64])

        #    b. Model architecture parameters
        kernLength = self.trial.suggest_categorical("kernLength", [64, 128, 256])
        F1 = self.trial.suggest_categorical("F1", [8, 16, 32])
        D = self.trial.suggest_categorical("D", [1, 2, 3])
        F2 = self.trial.suggest_categorical("F2", [16, 32, 64]) # F2 must be F1 * D
        hidden_dim = self.trial.suggest_categorical("hidden_dim", [64, 128, 256])
        layer_dim = self.trial.suggest_categorical("layer_dim", [1, 2, 3])
        dropout = self.trial.suggest_float("dropout", 0.1, 0.6)
        
        #    c. Optimizer parameters
        lr = self.trial.suggest_float("lr", 1e-4, 1e-2, log=True)

        # 2. Prepare the data using these parameters
        super()._prepare_data(is_trial=True, batch_size=batch_size, window_length=window_length)
        
        assert self.dataset is not None, "Dataset was not created correctly"
        n_electrodes = self.dataset.datasets[0].data[0].shape[0] # Get shape from underlying dataset

        # 3. Build the model and optimizer
        self.model = SSVEPClassifier(
            n_electrodes=n_electrodes, # Use value from data
            dropout=dropout,
            kernLength=kernLength,
            F1=F1,
            D=D,
            F2=F1 * D, # F2 is dependent on F1 and D
            hidden_dim=hidden_dim,
            layer_dim=layer_dim,
        ).to(self.device)
        
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=lr)

    # This method is called by train() for the final run
    def prepare_final_run(self):
        # 1. Get the best hyperparameters from the completed study
        study = self._get_study()
        best_params = study.best_params
        
        # 2. Prepare data using the best params
        super()._prepare_data(is_trial=False) # is_trial=False handles getting params from study
        
        assert self.dataset is not None, "Dataset was not created correctly"
        n_electrodes = self.dataset.datasets[0].data[0].shape[0]

        # 3. Build the final model and optimizer
        self.model = SSVEPClassifier(
            n_electrodes=n_electrodes,
            out_dim=2,
            dropout=best_params["dropout"],
            kernLength=best_params["kernLength"],
            F1=best_params["F1"],
            D=best_params["D"],
            F2=best_params["F1"] * best_params["D"],
            hidden_dim=best_params["hidden_dim"],
            layer_dim=best_params["layer_dim"],
        ).to(self.device)
        
        # Optional: Load pre-existing weights if you are resuming
        try:
            self.model.load_state_dict(torch.load(self.model_path))
            print(f"Loaded existing model weights from {self.model_path}")
        except Exception:
            print(f"No existing model weights found at {self.model_path}. Training from scratch.")
        
        lr = 0.00018182233882257615 # best_params["lr"]
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr)
        self.scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            self.optimizer,
            mode='max',        # “min” if you want to reduce lr when the quantity monitored has stopped decreasing
            factor=0.5,        # new_lr = lr * factor
            patience=20,        # number of epochs with no improvement after which lr will be reduced
            threshold=1e-4,    # threshold for measuring the new optimum, to only focus on significant changes
            threshold_mode='rel', # `'rel'` means compare change relative to best value. Could use `'abs'`.
            cooldown=0,        # epochs to wait before resuming normal operation after lr has been reduced
            min_lr=1e-6,       # lower bound on the lr
        )

trainer = CustomTrainer(
        data_path=data_path,
        optuna_db_path=optuna_db_path,
        model_path=model_path,
        train_epochs=500, # Final training epochs
        tune_epochs=50,   # Epochs per trial
        optuna_n_trials=50,
        task="mi",
        eeg_channels=eeg_channels,
        data_fraction=0.4
    )

In [ ]:
delete_existing = False
trainer.optimize(delete_existing)

In [ ]:
trainer.train()

In [ ]:
trainer._prepare_training(False)
trainer.model.eval()
f"test accuracy: {evaluate_model(trainer.model, trainer.eval_loader, device)}"